Для начала попробуем отправить запрос и проверить, что chatGpt правильно понимает задачу

In [13]:
from openai import OpenAI

client = OpenAI(
      api_key= 
)

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "Тебе необходимо сформировать математическую постановку задачи линейного программирования"},
    {"role": "user", "content": """Сформулируй математическую постановку задачи линейного программирования: Банк может открывать счета клиентов трех типов: физические лица, индивидуальный предприниматель, юридическое лицо. Ведение счетов для клиентов платное и составляет 7, 13 и 17 тыс. рублей в год соответственно. Нормативы расходования ресурсов сотрудников банка для ведения одного счета для клиента заданного типа составляют: 0,5; 0,8 и 1 час соответственно. Финансовые затраты для ведения счета для клиента заданного типа составляют: 0,6; 1,2; 1,5 тыс. руб. соответственно. Фонд человеко-часов сотрудников банка, выделенный для ведения счетов составляет 400 часов, а бюджет составляет 500 тыс. руб. За пределы фонда и объема бюджета, выделенного для ведения счетов клиентов всех типов выходить нельзя. Помогите банку спланировать распределение счетов для клиентов каждого типа с целью получения максимального дохода."""}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Пусть x1, x2 и x3 - количество счетов, открытых для физических лиц, индивидуальных предпринимателей и юридических лиц, соответственно.\n\nТогда математическая постановка задачи линейного программирования будет следующей:\n\nМаксимизировать функцию дохода:\n7 * x1 + 13 * x2 + 17 * x3\n\nС учетом нормативов расходования ресурсов сотрудников банка и финансовых затрат:\n0.5 * x1 + 0.8 * x2 + 1 * x3 <= 400 (Фонд человеко-часов)\n0.6 * x1 + 1.2 * x2 + 1.5 * x3 <= 500 (Бюджет)\n\nДополнительно требуется, чтобы количество счетов каждого типа было неотрицательным:\nx1 >= 0\nx2 >= 0\nx3 >= 0\n\nТаким образом, задача состоит в максимизации функции дохода при ограничениях по фонду человеко-часов и бюджету, а также учитывает неотрицательность количества счетов каждого типа.', role='assistant', function_call=None, tool_calls=None)


Отлично, теперь попросим вывести данные в формате нашей программы

In [14]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": 
        """Тебе необходимо сформировать математическую постановку задачи линейного программирования и записать ее в формате json. Пример твоего ответа:
{"W": [5,6,7],
"direction": "max",
"A": [
    [1.2, 0.8, 0.6],
    [0.5, 0.7, 1]
],
"constraints_direction": ["l", "l"],
"b": [500.0, 400.0],
"vars_constraints": [0.0, 0.0, 0.0]}
В этом примере: 
W - массив коэффициентов для целевой функции,
direction - направление оптимизации. Могут быть значения max и min,
A - двойной массив. Содержит массивы коэффициентов для ограничений,
constraints_direction - массив со знаками для ограничений. Могут быть значения: l - меньше, b - больше, e - равно,
b - массив правых частей ограничений,
vars_constraints - ограничения снизу для переменных."""},
    {"role": "user", "content": """Банк может открывать счета клиентов трех типов: физические лица, индивидуальный предприниматель, юридическое лицо. Ведение счетов для клиентов платное и составляет 7, 13 и 17 тыс. рублей в год соответственно. Нормативы расходования ресурсов сотрудников банка для ведения одного счета для клиента заданного типа составляют: 0,5; 0,8 и 1 час соответственно. Финансовые затраты для ведения счета для клиента заданного типа составляют: 0,6; 1,2; 1,5 тыс. руб. соответственно. Фонд человеко-часов сотрудников банка, выделенный для ведения счетов составляет 400 часов, а бюджет составляет 500 тыс. руб. За пределы фонда и объема бюджета, выделенного для ведения счетов клиентов всех типов выходить нельзя. Помогите банку спланировать распределение счетов для клиентов каждого типа с целью получения максимального дохода."""}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='{"W": [7, 13, 17],\n"direction": "max",\n"A": [\n    [0.5, 0.8, 1],\n    [0.6, 1.2, 1.5]\n],\n"constraints_direction": ["l", "l"],\n"b": [400.0, 500.0],\n"vars_constraints": [0.0, 0.0, 0.0]}', role='assistant', function_call=None, tool_calls=None)


Прекрасно! Получилось именно то что нужно.

In [16]:
completion.choices[0].message.content

'{"W": [7, 13, 17],\n"direction": "max",\n"A": [\n    [0.5, 0.8, 1],\n    [0.6, 1.2, 1.5]\n],\n"constraints_direction": ["l", "l"],\n"b": [400.0, 500.0],\n"vars_constraints": [0.0, 0.0, 0.0]}'

In [17]:
def get_input_data(text):
  completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": 
        """Тебе необходимо сформировать математическую постановку задачи линейного программирования и записать ее в формате json. Пример твоего ответа:
{"W": [5,6,7],
"direction": "max",
"A": [
    [1.2, 0.8, 0.6],
    [0.5, 0.7, 1]
],
"constraints_direction": ["l", "l"],
"b": [500.0, 400.0],
"vars_constraints": [0.0, 0.0, 0.0]}
В этом примере: 
W - массив коэффициентов для целевой функции,
direction - направление оптимизации. Могут быть значения max и min,
A - двойной массив. Содержит массивы коэффициентов для ограничений,
constraints_direction - массив со знаками для ограничений. Могут быть значения: l - меньше, b - больше, e - равно,
b - массив правых частей ограничений,
vars_constraints - ограничения снизу для переменных."""},
    {"role": "user", "content": text}
    ])
  return completion.choices[0].message.content

In [18]:
import json
import numpy as np
import copy

def read_file(name):
    with open(name, 'r') as f:
        data = f.read()

    return parse_input(data)
def parse_input(data):
    data = json.loads(data)
    if data['direction'] == 'min':
        c = np.array(data['W'], dtype=np.float64)
    else:
        c = - np.array(data['W'], dtype=np.float64)
    A = np.array(data['A'])
    b = np.array(data['b'])
    constr_dir = data['constraints_direction']
    unique, counts = np.unique(constr_dir, return_counts=True)
    c_dict = dict(zip(unique, counts))
    return data, A, b, c, constr_dir, c_dict
def make_canonical_form(data, A, constr_dir, c, c_dict):    
    A = np.hstack([A, np.zeros((len(A), c_dict['l']))])
    
    j = len(c)
    for i in range(len(constr_dir)):
        if data['constraints_direction'][i] == 'g':
            A[i, j] = -1
            j += 1
                
        elif data['constraints_direction'][i] == 'l':
            A[i, j] = 1
            j += 1
            
        elif data['constraints_direction'][i] == 'e':
            continue
        
        else:
            print('Wrong constraint direction')
            
    return A
def simplex_step(A_s, b_s, z, Q, base_vars, free_vars):
    main_col = np.argmin(z)
    
    
    penalty = 100_000
    row_array = []
    for i in range(len(A_s)):
        if A_s[i, main_col] > 0:
            row_array.append(b_s[i] / A_s[i, main_col])
        else:
            row_array.append(penalty)
            
    if np.sum(row_array) == penalty * len(A_s):
        print('Целевая функция не ограничена снизу')
        return 0
            
    else:
        main_row = np.argmin(row_array)
        main_el = A_s[main_row, main_col]
        
        
        #compute new matrix A 
        A_s_new = copy.deepcopy(A_s)
        A_s_new[main_row] = A_s[main_row] / main_el
        A_s_new[:, main_col] = -A_s[:, main_col]/main_el
        A_s_new[main_row, main_col] = 1/main_el
        
        for i in range(A_s.shape[0]):
            for j in range(A_s.shape[1]):
                if (i != main_row) and (j != main_col):
                    A_s_new[i, j] = A_s[i, j] - A_s[main_row, j] * A_s[i, main_col] / main_el
        
        
        #compute new z vector
        z_new = copy.deepcopy(z)
        z_new[main_col] = -z[main_col]/main_el
        for j in range(len(z_new)):
            if j != main_col:
                z_new[j] = z[j] - z[main_col] * A_s[main_row, j] / main_el
        
        
        #compute new b vector
        b_s_new = copy.deepcopy(b_s)
        b_s_new[main_row] = b_s[main_row]/main_el
        for i in range(len(b_s_new)):
            if i != main_row:
                b_s_new[i] = b_s[i] - b_s[main_row] * A_s[i, main_col] / main_el
        
        #compute cost function
        Q_new = Q - b_s[main_row] * z[main_col] / main_el
        
        #change base vars with free vars
        change_val = base_vars[main_row]
        base_vars[main_row] = free_vars[main_col]
        free_vars[main_col] = change_val
        
        
        return A_s_new, b_s_new, z_new, Q_new, base_vars, free_vars
def pretty_print(A_s, b_s, z, Q, base_vars, free_vars):
    print("---Begin---")
    print("A_s:")
    for row in A_s:
        print("  ", " ".join(f"{val:8.2f}" for val in row))
    
    print("\nb_s:")
    print("  ", " ".join(f"{val:8.2f}" for val in b_s))
    
    print("\nz:")
    print("  ", " ".join(f"{val:8.2f}" for val in z))
    
    print("\nQ:")
    print("  ", f"{Q:8.2f}")
    
    print("\nBase Variables:")
    print("  ", " ".join(map(str, base_vars)))
    
    print("\nFree Variables:")
    print("  ", " ".join(map(str, free_vars)))
    print("---End---")

Добавим чтение из файла:

In [19]:
with open('text_input.txt', 'r') as f:
  data = f.read()
get_input_data(data)
data, A, b, c, constr_dir, c_dict = parse_input(completion.choices[0].message.content)
A = make_canonical_form(data, A, constr_dir, c, c_dict)
b_s = b
A_s = A[:, :-2]
z = c
Q = 0
base_vars = np.arange(len(A_s[0]) + 1, len(A_s[0]) + 1 + len(b))
free_vars = np.arange(1, len(A_s[0]) + 1)
pretty_print(A_s, b_s, z, Q, base_vars, free_vars)
while np.any(z < 0):
    A_s, b_s, z, Q, base_vars, free_vars = simplex_step(A_s, b_s, z, Q, base_vars, free_vars)
    pretty_print(A_s, b_s, z, Q, base_vars, free_vars)
result = sorted(zip(base_vars, b_s), key=lambda x: x[0])
print(result)
result = [(key, value) for key, value in result if key <= 3]
result

---Begin---
A_s:
       0.50     0.80     1.00
       0.60     1.20     1.50

b_s:
     400.00   500.00

z:
      -7.00   -13.00   -17.00

Q:
       0.00

Base Variables:
   4 5

Free Variables:
   1 2 3
---End---
---Begin---
A_s:
       0.10     0.00    -0.67
       0.40     0.80     0.67

b_s:
      66.67   333.33

z:
      -0.20     0.60    11.33

Q:
    5666.67

Base Variables:
   4 3

Free Variables:
   1 2 5
---End---
---Begin---
A_s:
      10.00     0.00    -6.67
      -4.00     0.80     3.33

b_s:
     666.67    66.67

z:
       2.00     0.60    10.00

Q:
    5800.00

Base Variables:
   1 3

Free Variables:
   4 2 5
---End---
[(1, 666.6666666666666), (3, 66.66666666666669)]


[(1, 666.6666666666666), (3, 66.66666666666669)]

In [20]:
formatted_results = []

for item in result:
    variable_name = f"x_{item[0]}"
    variable_value = int(item[1])
    formatted_results.append(f"{variable_name}={variable_value}")

pretty_print = '; '.join(formatted_results)

print(pretty_print)

x_1=666; x_3=66
